In [1]:
import os, sys, re, string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from data_preparation import take_a_shower, clean_content_aravec

In [2]:
data = pd.read_csv('./data/data_v1.csv', encoding="utf-16")
data.sample(10)

,Username,Comment,is_off,off_word
13732,monsef jmili,ارحموا من في الأرض يرحمكم من في السماء,0,0
36898,Noura Rouis,حيوانات ومكروبات هدو هما صلاكط الي خارجين علي...,0,0
8931,ABStudio Pro,اللهم إني صائم .. خويا سعيد على وجه المتعة الل...,1,"0, وجه"
22499,قناة Safa et Marwa family,بارك من الضحك حسبيا الله ونعمة الكيل تلوزو اتم...,0,0
13328,Parakeets Budgie,كنت تنسول راسي باش هاد السيد عاجب الناس حتى شف...,0,0
10420,Ayati Aya,كل يبحت عن غايته,0,0
28987,ضوء القمر,khouya ntouma lharba ahssan chaine youtube f l...,0,0
34787,redazi1,Hada daba ghada ykabtouh yahwiwah wmin baad ym...,0,0
36752,Peachy bee,Clean 🔥🔥,0,0
33743,ali zawa,شكون متفق نصليو صلاة الجنازة على المغرب يسطاسي...,0,0


In [58]:
data_off = data.loc[data['is_off'] == 1, ['Comment', 'is_off']]
data_not_off = data.loc[data['is_off'] == 0, ['Comment', 'is_off']]

In [59]:
data_not_off = data_not_off.sample(n=3701)

In [4]:
data2 = pd.read_excel('./data/data_4000_selected.xlsx', engine='openpyxl')

In [5]:
data2.sample(10)

,Id,Platform,Comment,Majority_Label,Agreement,NumOfJudgementUsed,Total_Judgement,Vulgar:V/HateSpeech:HS/None:-
652,653,Twitter,@User.IDX آسف ع التعبير : .... إم الدنيا,Non-Offensive,66.666667,3,4,-
3264,3265,Twitter,@User.IDX اذا فيفي عبده الام المثاليه جت على هذي,Non-Offensive,100.000000,3,3,-
2852,2853,Facebook,انا كنت فى واصلى خلف أمام سعودى وكل صلاه يقرأ ...,Non-Offensive,100.000000,3,3,-
2397,2398,Twitter,@User.IDX الله يصبر امهاتهم ويربط ع قلوب اهالي...,Non-Offensive,100.000000,3,3,-
2870,2871,YouTube,انا اخر مرة واول مرة اروح صلالة قبل 3سنوات,Non-Offensive,100.000000,3,4,-
1707,1708,YouTube,ثلاث دقايق هذا الاسبوع قصيره بس خمس دقايق,Non-Offensive,100.000000,3,3,-
3424,3425,Twitter,@User.IDX الجزيرة تجاوزت مرحلة تشريد الشعوب إل...,Offensive,100.000000,3,3,-
180,181,Twitter,@User.IDX اللهم أهلك بني صهيون الانجاس الملعون...,Offensive,100.000000,3,4,HS
541,542,YouTube,فيني عاده من وانا صغير اذا حبيت شيء بقوه .. اح...,Non-Offensive,66.666667,3,3,-
830,831,YouTube,نأ نأ عندها شي يحطوه الهنود بانفهم مجوهرات شكل...,Non-Offensive,100.000000,3,3,-


In [6]:
data2_off = data2.loc[data2['Majority_Label'] == 'Offensive', ['Comment', 'Majority_Label']]
data2_not_off = data2.loc[data2['Majority_Label'] == 'Non-Offensive', ['Comment', 'Majority_Label']]

In [7]:
data2_off.columns = ['Comment', 'is_off']
data2_not_off.columns = ['Comment', 'is_off']

In [8]:
data2_off['is_off'] = 1
data2_not_off['is_off'] = 0

In [53]:
data2_not_off = data2_not_off.sample(n=675)

In [9]:
data_aj = pd.read_excel('./data/data_AJ_comments.xlsx', engine='openpyxl')

In [10]:
data_aj.sample(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,languagecomment,languagecomment:confidence,articletitle,body,bodylen,insdt,languagecomment_gold,link,serial,words
11865,1222286748,False,finalized,3,4/28/2017 14:40:39,-1,1.0000,"شرطة برلين تقتل ""متشددا"" هاجم شرطية",اذا عربت خربت و اخيرا اذا اكردت خربت!,37,2015-09-18 14:35:59.150,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,16835,8
20541,1222307637,False,finalized,3,5/1/2017 22:03:13,-1,0.6662,تغييرات واسعة بالسعودية ودعوة لمبايعة بن نايف ...,ما هذا الهبل و العبط الذى تقوم به. إحترم نفسك ...,60,2015-04-29 09:48:31.333,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,59447,12
8995,1222293644,False,finalized,3,4/29/2017 09:09:07,0,0.6603,فابيوس يشكك في تغيّر سريع لسياسة إيران,ماذا ربحت مصر من قطع العلاقات مع إيران 35 عام؟...,182,2015-07-30 17:27:11.740,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,31227,32
30907,1222292636,False,finalized,3,4/29/2017 04:26:10,-1,1.0000,تنظيم الدولة يتبنى تفجير أبها بالسعودية,تقود67 دوله لقصفهم وما تريدون الرد انتم من بدا...,180,2015-08-06 19:38:35.827,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,29240,30
5279,1222285521,False,finalized,3,4/28/2017 13:46:30,-1,0.6440,المقاومة تسيطر على المنصة الرئيسية بسد مأرب,التحالف يمضي الى الهزيمة بعد مشاهد بثتها فضائي...,159,2015-09-28 14:46:17.480,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,14284,25


In [41]:
data_aj_off = data_aj.loc[data_aj['languagecomment'] != 0, ['body', 'languagecomment']]
data_aj_not_off = data_aj.loc[data_aj['languagecomment'] == 0, ['body', 'languagecomment']]

In [42]:
data_aj_off.columns = ['Comment', 'is_off']
data_aj_not_off.columns = ['Comment', 'is_off']

In [43]:
data_aj_off['is_off'] = 1
data_aj_not_off['is_off'] = 0

In [54]:
data_aj_off = data_aj_off.sample(n=2624)
data_aj_not_off = data_aj_not_off.sample(n=2624)

In [55]:
print("MA  : " + str(len(data_off)) + "\nGolf: " + str(len(data2_off)) + "\nGolf: " + str(len(data_aj_off)))
print("Total Offensive Data : % 2d" %(len(data_off) + len(data2_off) + len(data_aj_off)))

MA  : 3701
Golf: 675
Golf: 2624
Total Offensive Data :  7000


In [46]:
dataset_offensive = pd.concat([data_off, data2_off, data_aj_off])

In [49]:
dataset_offensive.shape

(7000, 2)

In [60]:
print("MA  : " + str(len(data_not_off)) + "\nGolf: " + str(len(data2_not_off)) + "\nGolf: " + str(len(data_aj_not_off)))
print("Total Offensive Data : % 2d" %(len(data_not_off) + len(data2_not_off) + len(data_aj_not_off)))

MA  : 3701
Golf: 675
Golf: 2624
Total Offensive Data :  7000


In [69]:
dataset_not_offensive_all = pd.concat([data_not_off, data2_not_off, data_aj_not_off])

In [70]:
dataset_not_offensive_all

,Comment,is_off
21179,Ana walit kanchof had CHOF TV kaaatchwaah bnat...,0
8432,Ach had lmostawa arradi2 kalam sa9et lawah ou...,0
9292,المرجو قطع الانترنت على هاد البلاد وشكرا,0
29545,اويلي. على سؤال,0
8273,السلام عليكم أنا خولة كندير فيديوهات عن الحياة...,0
...,...,...
22565,اخي العزيز.. الأفضل أن تستخدم عبارة (نفوق ١٢٠٠...,0
13823,الاخ اللى يدعى ان رافال منزوعة الفعاليه ويدعى ...,0
26476,و اللذي نفسي بيده لن أتوقف عن مناصرتي و تئيدي ...,0
26313,الجيش والشرطه والشعب أيد واحده و تحيا مصر تحيا...,0


In [71]:
offensive_dataset = pd.concat([dataset_offensive, dataset_not_offensive_all]).sample(frac=1).reset_index(drop=True)

In [73]:
offensive_dataset

,Comment,is_off
0,شوف حد ين*** أنت و إيران.,1
1,شوف تيفي قوداتها مع الجن غي بلاتي يجي من مكة,0
2,عجب يتفاوظون على شعوب عربية وعليهم وهىلا تعلم,0
3,ههههههه هاد شوف تيفي اللي حطت عليه مشا فيها,0
4,الرزق يأتي من حيث لا تدري 🙌🏻❤️,0
...,...,...
13995,وأنشأ حنا مسلمين ولا كلاب . اس هادا اشي كيعلو...,1
13996,《 واذا قيل له اتق الله اخذته العزة بالاثم فحسب...,0
13997,الله يرضى عليكون زهران علوش وابو النصر متهمين ...,0
13998,الفلوس لي دفعوها خلصت\nو كله بثمنه\nعايزين تدخ...,0


In [89]:
offensive_dataset.loc[ 13851 , : ]

Comment    Merci mohamd sadise lhad zamla li nwadti flbla...
is_off                                                     0
Name: 13851, dtype: object

In [80]:
offensive_dataset.to_csv('./data/offensive_dataset.csv', index=False, encoding="utf-16")